In [1]:
import jieba
import pandas as pd 
from collections import Counter
from pyecharts.charts import Line,Pie,Scatter,Bar,Map,Grid
from pyecharts.charts import WordCloud
from pyecharts import options as opts
from pyecharts.globals import ThemeType
from pyecharts.globals import SymbolType
from pyecharts.commons.utils import JsCode

D:\rgzn\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\rgzn\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
D:\rgzn\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
df = pd.read_excel('旅游景点.xlsx')
df.head()

,城市,名称,星级,评分,价格,销量,省/市/区,坐标,简介,是否免费,具体地址
0,上海,上海迪士尼乐园,NaN,0.0,325.0,19459,上海·上海·浦东新区,"121.667917,31.149712",每个女孩都有一场迪士尼梦,False,上海市浦东新区川沙镇黄赵路310号上海迪士尼乐园
1,上海,上海海昌海洋公园,4A,0.0,276.5,19406,上海·上海·浦东新区,"121.915647,30.917713",看珍稀海洋生物 | 玩超刺激娱乐项目,False,上海市浦东新区南汇城银飞路166号
2,上海,上海野生动物园,5A,3.6,116.0,6764,上海·上海·浦东新区,"121.728112,31.059636",全球动物聚集地 | 零距离和动物做朋友,False,上海市浦东新区南六公路178号
3,上海,东方绿舟,4A,3.5,40.0,5353,上海·上海·青浦区,"121.015977,31.107866",全国首屈一指的青少年校外教育营地,False,上海市青浦区沪青平公路6888号
4,上海,东方明珠,5A,3.8,54.0,3966,上海·上海·浦东新区,"121.50626,31.245369",感受云端漫步，品味老上海风情,False,上海市浦东新区世纪大道1号


In [3]:
df = df[df['销量']!=0] 
df.shape

(2320, 11)

In [4]:
df['星级'].fillna('未知', inplace=True)
df.isnull().sum()

城市        0
名称        0
星级        0
评分        0
价格        0
销量        0
省/市/区     0
坐标        0
简介       37
是否免费      0
具体地址      2
dtype: int64

In [5]:
df.fillna('未知', inplace=True)
df.isnull().sum()

城市       0
名称       0
星级       0
评分       0
价格       0
销量       0
省/市/区    0
坐标       0
简介       0
是否免费     0
具体地址     0
dtype: int64

In [6]:
df.sort_values('销量', ascending=False).head()

,城市,名称,星级,评分,价格,销量,省/市/区,坐标,简介,是否免费,具体地址
0,上海,上海迪士尼乐园,未知,0.0,325.0,19459,上海·上海·浦东新区,"121.667917,31.149712",每个女孩都有一场迪士尼梦,False,上海市浦东新区川沙镇黄赵路310号上海迪士尼乐园
1,上海,上海海昌海洋公园,4A,0.0,276.5,19406,上海·上海·浦东新区,"121.915647,30.917713",看珍稀海洋生物 | 玩超刺激娱乐项目,False,上海市浦东新区南汇城银飞路166号
211,北京,故宫,5A,5.0,58.6,15277,北京·北京·东城区,"116.403347,39.922148",世界五大宫之首，穿越与您近在咫尺,False,北京市东城区景山前街4号
2187,陕西,秦始皇帝陵博物院（兵马俑）,5A,4.4,120.0,12714,陕西·西安·临潼区,"109.266029,34.386024",世界第八大奇迹,False,陕西省西安市临潼县秦始皇陵东1.5公里处
474,四川,成都大熊猫繁育研究基地,4A,4.0,52.0,9731,四川·成都·成华区,"104.152603,30.738951",无关黑与白， 不分胖与瘦， 可爱而又温暖,False,四川省成都市成华区外北熊猫大道1375号


In [151]:
def bar1():
    #渐变颜色代码
    color_js = """new echarts.graphic.LinearGradient(0, 0, 1, 1,
    [{offset: 0, color: '#3498DB'}, {offset: 1, color: '#6C3483'}], false)"""
    colors = ["#5793f3", "#d14a61", "#675bba"]
    #让列表内容从低到高进行排序
    sort_info = df.sort_values(by='销量', ascending=True)
    bar = (
        Bar()
        #传入X轴内容
        .add_xaxis(list(sort_info['名称'])[-20:])
        #Y轴内容，销量信息
        .add_yaxis(
            #图标名称
            series_name="热门景点销量",
            #内容
            y_axis=sort_info['销量'].values.tolist()[-20:],
            #轴的宽度
            category_gap="40%",
            yaxis_index=0,
            #渐变效果代码
            itemstyle_opts=opts.ItemStyleOpts(color=JsCode(color_js))
        )
        #传入第二个柱状图内容，价格
        .add_yaxis(
            series_name="热门景点价格", y_axis=sort_info['价格'].values.tolist()[-20:], yaxis_index=1, color=colors[0]
        )
        #传入第三个柱状图内容，评分
        .add_yaxis(
            series_name="热门景点评分", y_axis=sort_info['评分'].values.tolist()[-20:], yaxis_index=1, color=colors[0]
        )
        #设置数值列
        .extend_axis(
            #设置y轴刻度的最大值和最小值
            yaxis=opts.AxisOpts(
                type_="value",
                name="销量",
                min_=5000,
                max_=20000,
                #不显示出来
                is_show=False,
                #在图的右侧
                position="right",
                #刻度的颜色，其实不要也行
                axisline_opts=opts.AxisLineOpts(
                    linestyle_opts=opts.LineStyleOpts(color=colors[1])
                ),
                #刻度的内容，也是不要也行
                axislabel_opts=opts.LabelOpts(formatter="{value}"),
            )
        )
        
        .set_global_opts(
            #让x轴旋转45度，为了显示名称
            xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=45)),
            #y轴的刻度
            yaxis_opts=opts.AxisOpts(
                type_="value",
                name="销量",
                min_=5000,
                max_=20000,
                position="left",
                #传入渐变颜色效果
                axisline_opts=opts.AxisLineOpts(
                    linestyle_opts=opts.LineStyleOpts(color=JsCode(color_js))
                ),
                #y轴的值
                axislabel_opts=opts.LabelOpts(formatter="{value}"),
            ),
            tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        )
        #不显示数值内容
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    )
    return bar
# bar.render_notebook()


In [171]:
from pyecharts import options as opts
from pyecharts.charts import Map3D
from pyecharts.globals import ChartType
from pyecharts.commons.utils import JsCode

def map1():
    color_js = """new echarts.graphic.LinearGradient(0, 0, 1, 1,
    [{offset: 0, color: '#3498DB'}, {offset: 1, color: '#6C3483'}], false)"""
    #数值，名称 经纬度，值
    example_data = [
        ("黑龙江", [127.9688, 45.368, 4639]),
        ("内蒙古", [110.3467, 41.4899, 3959]),
        ("吉林", [125.8154, 44.2584, 3772]),
        ("辽宁", [123.1238, 42.1216, 10423]),
        ("河北", [114.4995, 38.1006, 6826]),
        ("天津", [117.4219, 39.4189, 5254]),
        ("山西", [112.3352, 37.9413, 15904]),
        ("陕西", [109.1162, 34.2004, 64353]),
        ("甘肃", [103.5901, 36.3043, 4338]),
        ("宁夏", [106.3586, 38.1775, 5622]),
        ("青海", [101.4038, 36.8207, 4591]),
        ("新疆", [87.9236, 43.5883, 3614]),
        ("西藏", [91.11, 29.97, 7028]),
        ("四川", [103.9526, 30.7617, 65052]),
        ("重庆", [108.384366, 30.439702, 20054]),
        ("山东", [117.1582, 36.8701, 32147]),
        ("河南", [113.4668, 34.6234, 33776]),
        ("江苏", [118.8062, 31.9208, 80783]),
        ("安徽", [117.29, 32.0581, 21027]),
        ("湖北", [114.3896, 30.6628, 22563]),
        ("浙江", [119.5313, 29.8773, 45481]),
        ("福建", [119.4543, 25.9222, 23256]),
        ("江西", [116.0046, 28.6633, 11046]),
        ("湖南", [113.0823, 28.2568, 6980]),
        ("贵州", [106.6992, 26.7682, 22499]),
        ("广西", [108.479, 23.1152, 37946]),
        ("海南", [110.3893, 19.8516, 44123]),
        ("云南", [102.710002, 25.045806, 28056]),
        ("上海", [121.4648, 31.2891, 84084]),
        ("北京", [116.4, 39.9, 93987]),
        ("台湾", [121.509062, 25.044332, 1001]),
        ("广东", [113.266530, 23.132191, 62757]),
        ("澳门", [113.549090, 22.198951, 3128]),
        ("香港", [114.173355, 22.320048, 1006]),
    ]


    c = (
        #3D图形
        Map3D()
        .add_schema(
            #地图背景颜色代码
            itemstyle_opts=opts.ItemStyleOpts(
                color="rgb(5,101,123)",
                opacity=1,
                border_width=0.8,
                border_color="rgb(62,215,213)",
            ),
            #内容的标签
            map3d_label=opts.Map3DLabelOpts(
                is_show=False,
                formatter=JsCode("function(data){return data.name + " " + data.value[2];}"),
            ),
            #标签的大小
            emphasis_label_opts=opts.LabelOpts(
                is_show=False,
                color="#fff",
                font_size=10,
                background_color="rgba(0,23,11,0)",
            ),
            #颜色亮度
            light_opts=opts.Map3DLightOpts(
                main_color="#fff",
                main_intensity=1.2,
                main_shadow_quality="high",
                is_main_shadow=False,
                main_beta=10,
                ambient_intensity=0.3,
            ),
        )
        #绘制3D地图
        .add(
            #标签名称
            series_name="假期出行分布",
            #数值
            data_pair=example_data,
            #图的类型
            type_=ChartType.BAR3D,
            #柱子的大小
            bar_size=1,
            shading="lambert",
            #名称
            label_opts=opts.LabelOpts(
                is_show=False,
                formatter=JsCode("function(data){return data.name + ' ' + data.value[2];}"),
            ),
        )
        #标题
        .set_global_opts(title_opts=opts.TitleOpts(title="假期出行数据地图分布"))
    )
    return c

# c.render_notebook()

In [176]:
from pyecharts.charts import Page
#组合工具
def page_default_layout():
    page = Page(layout=Page.DraggablePageLayout)
    page.add(
        bar1(),
        map1()
    )
    page.save_resize_html(cfg_file="chart_config.json")
    return page
#     page.render()
#     return page
# #把这两个图形，结合到一起呈现一个大屏效果
page = page_default_layout()
page.render_notebook()